# SdG based on phoneme generation and text-to-speech (TTS)

In [16]:
from espeak_phonemizer import Phonemizer
import numpy as np
from collections import defaultdict
import random

In [2]:
def train_markov_model(phoneme_sequences, n=2):
    """Train an n-gram Markov chain on a list of phoneme sequences."""
    transitions = defaultdict(lambda: defaultdict(int))
    for seq in phoneme_sequences:
        tokens = seq.split()
        for i in range(len(tokens) - n):
            prefix = tuple(tokens[i:i+n-1])
            next_token = tokens[i+n-1]
            transitions[prefix][next_token] += 1
    # Normalize to probabilities
    model = {
        prefix: {k: v / sum(d.values()) for k, v in d.items()}
        for prefix, d in transitions.items()
    }
    return model

def sample_markov(model, n=2, max_len=50, temperature=1.0):
    """Generate a random phoneme sequence."""
    prefix = random.choice(list(model.keys()))
    result = list(prefix)
    for _ in range(max_len - n):
        next_probs = model.get(prefix)
        if not next_probs:
            break
        tokens, probs = zip(*next_probs.items())
        # Apply temperature for randomness control
        probs = np.array(probs) ** (1/temperature)
        probs /= probs.sum()
        next_token = np.random.choice(tokens, p=probs)
        result.append(next_token)
        prefix = tuple(result[-(n-1):])
    return " ".join(result)


## Test on sample texts

In [19]:
sample_texts = [
    "Das Wetter ist heute schön.",
    "Ich gehe gerne im Wald spazieren.",
    "Die Sonne scheint über den Bergen.",
    "Wir trinken Kaffee am Nachmittag."
]

P = Phonemizer()
phonemes = [P.phonemize(text, voice='de') for text in sample_texts]  # Example usage

for t, p in zip(sample_texts, phonemes):
    print(t)
    print(p)
    print()

    
# 1. Collect phoneme sequences
phoneme_sequences = phonemes  # from step 2

# 2. Train Markov model
model = train_markov_model(phoneme_sequences, n=3)  # trigram

# 3. Sample
for temp in [0.7, 1.0, 1.5]:
    print(f"\nTemperature={temp}")
    print(sample_markov(model, n=3, temperature=temp))


Das Wetter ist heute schön.
das vˈɛtɜ ɪst hˈɔøtə ʃˈøːn

Ich gehe gerne im Wald spazieren.
ɪç ɡˈeːə ɡˈɛɾnə ɪm vˈalt ʃpatsˈiːrən

Die Sonne scheint über den Bergen.
diː zˈɔnə ʃˈaɪnt ˌyːbɜ deːn bˈɛɾɡən

Wir trinken Kaffee am Nachmittag.
viːɾ tɾˈɪŋkən kˈafeː am nˈaxmɪtˌɑːk


Temperature=0.7
ɪç ɡˈeːə ɡˈɛɾnə ɪm vˈalt

Temperature=1.0
ɡˈɛɾnə ɪm vˈalt

Temperature=1.5
zˈɔnə ʃˈaɪnt ˌyːbɜ deːn
